In [4]:
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_debug
from langchain_core.globals import set_llm_cache
from langchain_core.globals import set_verbose

set_debug(False)
set_verbose(False)
set_llm_cache(InMemoryCache())

In [ ]:
!pip install wikipedia

In [ ]:
# 使用集成的Wikipedia工具
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.invoke({"query": "langchain"}))

print(f"Name: {tool.name}")
print(f"Description: {tool.description}")
print(f"args schema: {tool.args}")
print(f"returns directly?: {tool.return_direct}")

In [ ]:
# 定义一个工具
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


@tool("multiplication-tool", args_schema=CalculatorInput, return_direct=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiply.invoke({"a": 4, "b": 3})

In [14]:
#定义1个工具
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama

class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

print(calculator.name)
print(calculator.description)
print(calculator.args)

#调用工具
print(calculator.invoke({"a": 11, "b": 3}))
# 
tools = [calculator]


llm = ChatOllama(
    model="qwen2:7b",
    temperature=0,
    # other params...
)
llm_with_tools = llm.bind_tools(tools)
query = "What is 3 * 12?"
#query = "What is 3 * 12? Also, what is 11 + 49?"
from langchain_core.messages import HumanMessage

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(query)
print(ai_msg.tool_calls)

messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    # 选择匹配的工具调用
    selected_tool = next((tool for tool in tools if tool.name == tool_call['name']), None)
    if selected_tool:
        tool_msg = selected_tool.invoke(tool_call['args'])
        # 调用结果重写传递给llm,生成最终的回答
        messages.append(HumanMessage(content=str(tool_msg)))
        llm_with_tools.invoke(messages)


Calculator
multiply numbers
{'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}
33
[{'name': 'Calculator', 'args': {'a': 3, 'b': 12}, 'id': '5fe95e50-b1f1-4fe5-b080-099605d2e740', 'type': 'tool_call'}]
36


AIMessage(content='The result of multiplying 3 by 12 is 36.', additional_kwargs={}, response_metadata={'model': 'qwen2:7b', 'created_at': '2024-11-21T04:42:51.252299Z', 'message': {'role': 'assistant', 'content': 'The result of multiplying 3 by 12 is 36.'}, 'done_reason': 'stop', 'done': True, 'total_duration': 978726375, 'load_duration': 309337834, 'prompt_eval_count': 196, 'prompt_eval_duration': 329000000, 'eval_count': 16, 'eval_duration': 197000000}, id='run-601da4d6-32d1-4fd8-9cf9-78f1eada40d5-0', usage_metadata={'input_tokens': 196, 'output_tokens': 16, 'total_tokens': 212})